In [1]:
from google.colab import files
uploaded = files.upload()

Saving 100_Unique_QA_Dataset.csv to 100_Unique_QA_Dataset.csv


In [2]:
import pandas as pd

df = pd.read_csv("100_Unique_QA_Dataset.csv")
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [3]:
#tokenization

def tokenize(text):
  text = text.lower()
  text = text.replace("?", " ")
  text = text.replace("'", " ")
  return text.split()

In [4]:
tokenize("my Name 'Satyam'?")

['my', 'name', 'satyam']

In [5]:
vocab = {'<UNK>': 0}

In [32]:
def build_vocab(row):
  tokenized_q = tokenize(row['question'])
  tokenized_a = tokenize(row['answer'])
  merged_tokens = tokenized_q + tokenized_a

  for token in merged_tokens:
    if token not in vocab:
      vocab[token] = len(vocab)

In [7]:
df.apply(build_vocab, axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [8]:
vocab

{'<UNK>': 0,
 'what': 1,
 'is': 2,
 'the': 3,
 'capital': 4,
 'of': 5,
 'france': 6,
 'paris': 7,
 'germany': 8,
 'berlin': 9,
 'who': 10,
 'wrote': 11,
 'to': 12,
 'kill': 13,
 'a': 14,
 'mockingbird': 15,
 'harper-lee': 16,
 'largest': 17,
 'planet': 18,
 'in': 19,
 'our': 20,
 'solar': 21,
 'system': 22,
 'jupiter': 23,
 'boiling': 24,
 'point': 25,
 'water': 26,
 'celsius': 27,
 '100': 28,
 'painted': 29,
 'mona': 30,
 'lisa': 31,
 'leonardo-da-vinci': 32,
 'square': 33,
 'root': 34,
 '64': 35,
 '8': 36,
 'chemical': 37,
 'symbol': 38,
 'for': 39,
 'gold': 40,
 'au': 41,
 'which': 42,
 'year': 43,
 'did': 44,
 'world': 45,
 'war': 46,
 'ii': 47,
 'end': 48,
 '1945': 49,
 'longest': 50,
 'river': 51,
 'nile': 52,
 'japan': 53,
 'tokyo': 54,
 'developed': 55,
 'theory': 56,
 'relativity': 57,
 'albert-einstein': 58,
 'freezing': 59,
 'fahrenheit': 60,
 '32': 61,
 'known': 62,
 'as': 63,
 'red': 64,
 'mars': 65,
 'author': 66,
 '1984': 67,
 'george-orwell': 68,
 'currency': 69,
 'unit

In [9]:
def text_to_indices(text, vocab):
  indexed_text = []

  for token in tokenize(text):
    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])

  return indexed_text

In [10]:
text_to_indices("what is ADIT?", vocab)

[1, 2, 0]

In [11]:
import torch
from torch.utils.data import Dataset, DataLoader

In [12]:
class QADataset(Dataset):

  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, index):
    numerical_question = text_to_indices(self.df.iloc[index]['question'], self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)

    return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [13]:
dataset = QADataset(df, vocab)

In [14]:
dataloader = DataLoader(dataset, batch_size = 1, shuffle=True)

In [16]:
for question, answer in dataloader:
  print(question, answer[0])

tensor([[  1,   2,   3, 221,   5, 222, 223, 224]]) tensor([225])
tensor([[ 10, 140,   3, 141, 171,   5,   3,  70, 172]]) tensor([173])
tensor([[  1,   2,   3,   4,   5, 236, 237]]) tensor([238])
tensor([[ 1,  2,  3,  4,  5, 53]]) tensor([54])
tensor([[  1,   2,   3,   4,   5, 279]]) tensor([280])
tensor([[ 42, 200,   2,  14, 201, 202, 203, 204]]) tensor([205])
tensor([[ 42, 216, 118, 217, 218,  19,  14, 219,  43]]) tensor([220])
tensor([[ 42,   2,   3, 210, 137, 168, 211, 169]]) tensor([113])
tensor([[ 1,  2,  3, 92, 93, 94]]) tensor([95])
tensor([[  1,   2,   3,   4,   5, 113]]) tensor([114])
tensor([[42, 86, 87, 88, 89, 39, 90]]) tensor([91])
tensor([[1, 2, 3, 4, 5, 6]]) tensor([7])
tensor([[  1,   2,   3,  37,  38,  39, 161]]) tensor([162])
tensor([[  1,   2,   3, 103,   5, 104,  19, 105]]) tensor([106])
tensor([[ 42, 107,   2, 108,  19, 109]]) tensor([110])
tensor([[  1,   2,   3,  69,   5, 155]]) tensor([156])
tensor([[ 10,  75, 208]]) tensor([209])
tensor([[ 10,  75,   3, 296,  1

In [17]:
import torch.nn as nn

In [18]:
class SimpleRNN(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_question = self.embedding(question)
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))

    return output

In [19]:
learning_rate = 0.001
epochs = 20

In [20]:
model = SimpleRNN(len(vocab))

In [22]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [23]:
for epoch in range(epochs):
  total_loss = 0

  for question, answer in dataloader:
    optimizer.zero_grad()
    output = model(question)
    loss = criterion(output, answer[0])

    loss.backward()
    optimizer.step()
    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 524.707383
Epoch: 2, Loss: 455.285334
Epoch: 3, Loss: 375.089374
Epoch: 4, Loss: 315.002681
Epoch: 5, Loss: 263.751134
Epoch: 6, Loss: 216.107218
Epoch: 7, Loss: 172.681464
Epoch: 8, Loss: 135.093280
Epoch: 9, Loss: 104.182101
Epoch: 10, Loss: 79.702625
Epoch: 11, Loss: 61.705836
Epoch: 12, Loss: 48.772260
Epoch: 13, Loss: 38.249917
Epoch: 14, Loss: 30.990609
Epoch: 15, Loss: 25.237373
Epoch: 16, Loss: 20.947375
Epoch: 17, Loss: 17.474791
Epoch: 18, Loss: 14.869757
Epoch: 19, Loss: 12.767282
Epoch: 20, Loss: 11.113978


In [24]:
def predict(model, question, threshold=0.5):
  numerical_question = text_to_indices(question, vocab)
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)
  output = model(question_tensor)
  probs = torch.nn.functional.softmax(output, dim=1)
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print("I don't know")

  print(list(vocab.keys())[index])

In [31]:
predict(model, "Who has the largest dick in our solar system?")

jupiter
